In [19]:
import pennylane as qml
import tensorflow as tf
import networkx as nx
import random

tf.keras.backend.set_floatx("float64")

In [3]:
wires = 5
layers = 8
dev1 = qml.device("default.qubit", wires = wires)
dev2 = qml.device("default.qubit", wires = wires)
gate_set = [qml.RX, qml.RY, qml.RZ]

## Define Problem Graph

In [26]:
def dregular_graph(n,d,mu,sigma):
    """
        n --> number of qubits
        d --> number of edges connected to one node (n x d must be an even number)
    """
    G= nx.generators.random_graphs.random_regular_graph(d,n)
    weights = []
    for e in list(G.edges):
        w = round(random.gauss(mu,sigma),2)
        G[e[0]][e[1]]["weight"] = w
        weights.append(w)
    return G, np.array(weights)

def adjacency_matrix(G):
    adj = np.zeros((len(G.nodes),len(G.nodes)))
    for edge in G.edges:
        i = edge[0]
        j = edge[1]
        adj[i,j] = 1
        adj[j,i] = 0
    return adj

G, weights = dregular_graph(wires,2,0,2)
edges = list(G.edges)
Adj = adjacency_matrix(G)
Adj = tf.constant(Adj)

In [27]:
obs = [qml.PauliZ(e[0])@qml.PauliZ(e[1]) for e in edges]# Define VQE opeartors
coeffs = weights  # Coefficients of MaxCut Problem

## Define Circuits for sampling and for VQE

In [28]:
def circuit_VQE(params, wires, num_qubits=4, random_gate_sequence=None, layers=1):
    for i in range(num_qubits):
        qml.RY(np.pi/4, wires=i)
    for j in range(layers):
        for i in range(num_qubits):
            random_gate_sequence[j][i](params[j][i], wires=i)
        for i in range(num_qubits):
            qml.CZ(wires = [i, (i+1)%num_qubits])

def circuit_with_NN(params, num_qubits=4, random_gate_sequence=None, layers=1):
    for i in range(num_qubits):
        qml.RY(np.pi/4, wires=i)
    for j in range(layers):
        for i in range(num_qubits):
            random_gate_sequence[j][i](params[j][i], wires=i)
        for i in range(num_qubits):
            qml.CZ(wires = [i, (i+1)%num_qubits])
    wirelist = [i for i in range(num_qubits)]
    return [qml.sample(qml.PauliZ(i)) for i in range(num_qubits)]

## Define Cost for VQE

In [29]:
gate_sequence = [[np.random.choice(gate_set) for i in range(wires)] for j in range(layers)]

# H = qml.vqe.Hamiltonian(coeffs, obs)

# cost = qml.VQECost(circuit_VQE, H, dev1, interface="tf")

## Define Cost for Samples

In [30]:
def cost_for_single_sample(samples, coeffs, adjacency_matrix):
    S = tf.transpose(samples)
    B = tf.linalg.matvec(adjacency_matrix, S[:, 0])
    A = samples[:,0]*coeffs
    return tf.tensordot(A,B,1)

def cost_all_samples(samples, coeffs, adjacency_matrix):
    A = tf.multiply(samples, coeffs)
    B = tf.transpose(tf.linalg.matmul(adjacency_matrix, samples, transpose_b=True))
    return tf.reduce_sum(tf.multiply(A,B), 1)

## Define NN

In [31]:
clayer1 = tf.keras.layers.Dense(wires)
clayer2 = tf.keras.layers.Dense(10, activation="relu")
clayer3 = tf.keras.layers.Dense(wires, activation="tanh")
model = tf.keras.models.Sequential([clayer1, clayer2, clayer3])

In [32]:
def loss_NN(model, x):
    out = model(x)
    E = cost_all_samples(out, coeffs, adjacency_matrix)
    return tf.reduce_mean(E)

In [33]:
def gradient_NN(model, inputs):
    with tf.GradientTape() as tape:
        loss_value = loss_NN(model, inputs)
    return loss_value, tape.gradient(loss_value, model.trainable_variables)

## Define QAOA-NN hybrid


### Try to do single shot QAOA folowed by NN and then do gradients manually over QAOA

In [159]:
dev = qml.device("default.qubit", wires = wires, analytic = False, shots = 1)

def circuit_single_shot(params, num_qubits=wires, random_gate_sequence=None, layers=1):
    for i in range(num_qubits):
        qml.RY(np.pi/4, wires=i)
    for j in range(layers):
        for i in range(num_qubits):
            random_gate_sequence[j][i](params[j][i], wires=i)
        for i in range(num_qubits):
            qml.CZ(wires = [i, (i+1)%num_qubits])
    return [qml.expval(qml.PauliZ(i)) for i in range(num_qubits)]

In [160]:
qcircuit = qml.QNode(circuit_single_shot, dev).to_tf()

In [183]:
def forward_pass(params):
    sample = q_circuit(params, num_qubits=wires, random_gate_sequence=gate_sequence, layers=1)
    sample = tf.reshape(sample, (1,5))
    X = tf.constant(sample)
    out = model(X)
    return out

In [192]:
params = tf.Variable(np.random.rand(layers, wires), dtype=tf.float64)
out = forward_pass(params)
cost_for_single_sample(out, coeffs, Adj), out

(<tf.Tensor: shape=(), dtype=float64, numpy=0.9259328820500101>,
 <tf.Tensor: shape=(1, 5), dtype=float64, numpy=array([[-0.54549323, -0.19957194, -0.23025902,  0.12564628, -0.63428352]])>)

In [221]:
def manual_gradient(params):
    P = params.numpy()
    eps = np.pi/2
    shape = P.shape
    gradient = np.zeros(shape)
    for i in range(shape[0]):
        for j in range(shape[1]):
            eps_plus = P.copy()
            eps_plus[i][j] += eps
            out = forward_pass(eps_plus)
            exp_value_plus = cost_for_single_sample(out, coeffs, Adj)

            eps_minus = P.copy()
            eps_minus[i][j] -= eps
            out = forward_pass(eps_minus)
            exp_value_minus = cost_for_single_sample(out, coeffs, Adj)
            
            gradient_result = 0.5 * (exp_value_plus - exp_value_minus)
            
            gradient[i][j] = gradient_result
    return gradient

In [228]:
manual_gradient(params)

array([[ 0.2361364 ,  0.21948676,  1.1701787 ,  0.34196377, -0.1068798 ],
       [ 0.04431746,  0.38075748,  0.1524103 ,  0.09015744, -0.38850179],
       [-0.38780375, -0.1524103 ,  0.41070001, -0.19120401, -0.29764631],
       [-0.24256774,  0.34196377,  0.21605505, -0.09015744, -0.45557825],
       [-0.23829401,  0.        , -0.72721505, -0.2361364 ,  0.00643134],
       [-0.24256774, -0.19459336,  0.14950974, -0.56440482,  0.38780375],
       [-0.09015744,  0.19459336, -0.73805749,  0.81465392, -0.09015744],
       [ 0.23905379, -0.0869474 , -0.38780375, -0.69814599, -0.35818678]])

## Summ over gradients

For this single shot gradient we can average over many gradients coming from single shots with the same parameters to reduce the stochasticity.

Therefore it probably would make sense to work with the sampling output of pennylane so we can do tensor contractions instead of loops

In [34]:
qcircuit_with_NN = qml.QNode(circuit_with_NN, dev1).to_tf()

def forward_pass_samples(params):
    samples = qcircuit_with_NN(params, random_gate_sequence=gate_sequence, layers=layers, num_qubits=wires)
    samples =  tf.transpose(tf.cast(samples, tf.float64, name=None))
    X = tf.constant(samples)
    out = model(X)
    return tf.reduce_mean(cost_all_samples(out, coeffs, Adj))

params = tf.Variable(np.random.rand(layers, wires), dtype=tf.float64)
forward_pass_samples(params)

<tf.Tensor: shape=(), dtype=float64, numpy=-0.14357024365503693>

In [35]:
def manual_gradient_samples(params):
    P = params.numpy()
    eps = np.pi/2
    shape = P.shape
    gradient = np.zeros(shape)
    for i in range(shape[0]):
        for j in range(shape[1]):
            eps_plus = P.copy()
            eps_plus[i][j] += eps
            exp_value_plus = forward_pass_samples(eps_plus)
    
            eps_minus = P.copy()
            eps_minus[i][j] -= eps
            exp_value_minus = forward_pass_samples(eps_minus)
            
            gradient_result = 0.5 * (exp_value_plus - exp_value_minus)
            
            gradient[i][j] = gradient_result
    return gradient

In [36]:
manual_gradient_samples(params)

array([[ 0.00131634,  0.00283693, -0.07341341,  0.10757908, -0.05456212],
       [ 0.01706887,  0.03055452, -0.11010151,  0.04979852,  0.08616391],
       [ 0.02247844,  0.03400669,  0.02290727,  0.0688217 , -0.03284166],
       [ 0.02259739,  0.04393828, -0.0253544 ,  0.04535229,  0.10176035],
       [ 0.02351618,  0.03694815,  0.0053918 , -0.04880971, -0.02670245],
       [-0.00416254,  0.02007619, -0.00053258,  0.0671948 ,  0.03127292],
       [ 0.00228802, -0.00400606, -0.00365318,  0.0398405 ,  0.02187599],
       [-0.03619686,  0.02033508, -0.01646239,  0.00780342,  0.07866213]])

## Train full model

In [37]:
def gradient_NN():
    with tf.GradientTape() as tape:
        loss_value = forward_pass_samples(params)
    return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [38]:
lr = 0.01
optimizer = tf.keras.optimizers.SGD(learning_rate=lr)

params = tf.Variable(np.random.rand(layers, wires), dtype=tf.float64)
model = tf.keras.models.Sequential([clayer1, clayer2, clayer3])

for i in range(500):
    gr = manual_gradient_samples(params)
    params.assign_sub(lr*gr)
    loss_value, grads = gradient_NN()
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    print(loss_value.numpy())

-0.19804944178805953
-0.23349043507578693
-0.27933413674813373
-0.3112770938103706
-0.3335502711705086
-0.34842174187260055
-0.4171131908050715
-0.41168615302635386
-0.4693125960132616
-0.4669137826723878
-0.5270703720605201
-0.5545213932660653
-0.5590826275314382
-0.593508580933411
-0.5974493315621221
-0.700446479930616
-0.7425824542906219
-0.7096366912941025
-0.7605003574134135
-0.7745210384667869
-0.8631388596272928
-0.8706516376700941
-0.8979616854873802
-0.9746367184921197
-1.0051804590997353
-1.034384507946163
-1.0442784355808685
-1.1117552800731931
-1.1469512083547295
-1.1965773408301916
-1.2455999851280595
-1.295191797069446
-1.3409501718747086
-1.3810953319418882
-1.426709033889099
-1.4922304913059266
-1.5781248875059364
-1.6254950439480356
-1.6868737307173096
-1.806356980238592
-1.9237815455338896
-1.9629194123509124
-2.1286195507835446
-2.2764766968827423
-2.4850834437500833
-2.6686548659503586
-2.92585073595426
-3.2293585589851217
-3.3901371817307617
-3.7124318715172517
-4.

KeyboardInterrupt: 